<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd 

<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [ ]:
df = pd.read_csv("gdrive/My Drive/mbti/all_users_v2.csv", sep = ';', header = 0) 

In [ ]:
df

# Pre-Processing

<span style="font-size:1.1em;"> Entrylere ön işleme adımları uygulanır. Bu adımlar:</span>


1. <span style="font-size:1.1em;">Bütün harflerin küçük harf haline getirilmesi</span>


2. <span style="font-size:1.1em;">Entrylerden stop words'ler silinmelidir.</span>


3. <span style="font-size:1.1em;">Web sitelerinin temizlenmesi</span>


4. <span style="font-size:1.1em;">Noktalama işaretlerinin temizlenmesi</span>


5. <span style="font-size:1.1em;">Rakamların temizlenmesi</span>


6. <span style="font-size:1.1em;">Stemming kullanılması</span>


<span style="font-size:1.1em;">    
Temizlenme işlemlerinde empty string yerine space (' ') ile replace edilerek yapılmalıdır. Arından da fazla boşluklar vs trim edilmelidir.

```[+-]?([0-9]*)([.][0-9])?``` regex olarak kullanılacak hem rakamların silinmesinde hem de ondaklı sayıların silinmesinde --> buna gerek kalmadı

</span>


<span style="font-size:1.1em;"> Stop words, drive'da /mbti altındaki stop_words_tr.txt okunarak alınır</span>



In [ ]:
file = open("gdrive/My Drive/mbti/stop_words_tr.txt")
stop_word_list = file.read().split('\n')
file.close()

In [ ]:
stop_word_list

<span style="font-size:1.1em">Entrylerdeki harfler küçük harf haline getirilir</span>

In [ ]:
df['entry'] = df['entry'].str.lower()
df['entry']

In [ ]:
df['entry'] = df['entry'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_word_list)]))
df['entry']


<span style="font-size:1.1em;">Entrylerden web site linkleri silinir.</span>

In [ ]:
df['entry'] = df['entry'].str.replace(r'http\S+', ' ', regex=True)
df['entry'] = df['entry'].str.replace(r'www\S+', '', regex=True)
df['entry']

<span style="font-size:1.1em;">Entrylerden noktalama işaretleri temizlenir.</span>

In [ ]:
import string

df['entry'] = df['entry'].apply(lambda x: x.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))))
df['entry']

Entrylerden rakamlar silinir.

In [ ]:
df['entry'] = df['entry'].str.replace('\d+', ' ')
df['entry']

Verisetinin stemmera sokmadan önceki ön işlemeden geçirilmiş hali kaydedilir. Buradaki amaç stemmerdan geçirilmiş ve geçirilmemiş halinin başarı açısından karşılaştırılmasının yapılabilmesi.

In [ ]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv", sep=';', encoding='utf-8')

SnowballStemmer içerisinde Türkçe dili için kullanabileceğimiz TurkishStemmer import edilir. Stemmer ile kelimelerin köklerine ulaşılır. Lemmatization işlemiyle kök bulunduğu zaman elde edilen kökler biçimsel açıdan da doğru olurken (yani mantıksal açıdan da) stemmer ile elde edilen köklerde böyle bir durum söz konusu değildir. Dolayısıyla stemming edilmis hallerine bakıldıgında mantıksal açıdan düzgün sonuclar vermeyebilir.

Google Colab'a snowballstemmer ile ilgil package'i indirmek için aşağıdaki komutu çalıştırıyoruz.

In [ ]:
!pip install snowballstemmer

In [ ]:
from snowballstemmer import TurkishStemmer

In [ ]:
stemmer = TurkishStemmer()
df['entry']

In [ ]:
indexes = df.index

for i in range(len(indexes)):
    index = indexes[i]
    entry = df['entry'][index]
    entry_kokler = stemmer.stemWords(entry.split(" "))
    df['entry'][index] = " ".join(entry_kokler)


In [ ]:
df['entry']

Stemmera sokulmuş hali kaydedilir.

In [ ]:
df.to_csv("gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv", sep=';', encoding='utf-8')